<a href="https://colab.research.google.com/github/pscabral/IFMASOFTEX_1/blob/main/Yolo8_v1.8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mega.py

from mega import Mega

# Crie uma instância da classe Mega
mega = Mega()

# Cole o link completo do Mega.nz (incluindo a chave)
url = "https://mega.nz/file/gzcmkBbS#Xo0iwU47Ffld8w4sqtNYtDkjh5vmSud05X_xqLEjVGw"

# Use o link completo para fazer o download
mega.download_url(url)

!unzip Comida2.0.v1i.yolov8.zip

In [ ]:
!pip install ultralytics

In [ ]:
!pip install albumentations

In [4]:
import os
import random
import shutil
from pathlib import Path

import pandas as pd
from PIL import Image
import cv2
from ultralytics import YOLO
from IPython.display import Video
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import trange, tqdm
import warnings
import glob
import albumentations as A
import matplotlib.pyplot as plt
from shutil import copyfile
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [ ]:
def transformer(image_folder_path, annotation_folder_path):
    image_folder = Path(image_folder_path)
    annotation_folder = Path(annotation_folder_path)

    for image_path in image_folder.glob('**/*.jpg'):
        image_name = image_path.stem
        relative_label_path = image_path.relative_to(image_folder).with_suffix('.txt')
        label_path = annotation_folder / relative_label_path

        try:
            print(f'Processando: {image_name}')

            if not label_path.exists():
                print(f'Aviso: O arquivo de anotação não existe para {image_name}')
                continue

            horizontal_flip(image_path, label_path, image_name)
            vertical_flip(image_path, label_path, image_name)

            print(f'{image_name} processado com sucesso')

        except Exception as e:
            print(f'Falhou: {image_name}. Erro: {e}')

def horizontal_flip(image_path, label_path, save_name):
    try:
        print(f'  Aplicando transformação horizontal para: {image_path}')

        image = cv2.imread(str(image_path))

        with open(label_path, 'r') as f:
            lines = f.readlines()

        bbox = []
        for line in lines:
            l = line.split(' ')
            bbox.append([float(l[1]), float(l[2]), float(l[3]), float(l[4]), str(l[0])])

        transform = A.Compose([A.HorizontalFlip()], bbox_params=A.BboxParams(format='yolo'))
        transformed = transform(image=image, bboxes=bbox)

        image_transformed = transformed['image']
        boxes = transformed['bboxes']

        save_image_path = image_path.parent / f'{save_name}_hf.jpg'
        cv2.imwrite(str(save_image_path), image_transformed)

        save_label_path = label_path.parent / f'{save_name}_hf.txt'
        with open(str(save_label_path), 'w') as f:
            for box in boxes:
                f.write(f"{box[-1]} {box[0]} {box[1]} {box[2]} {box[3]}\n")

        print(f'  Transformação concluída para: {image_path}')

    except Exception as e:
        raise RuntimeError(f'Falhou: {save_name}. Erro: {e}')

def vertical_flip(image_path, label_path, save_name):
    try:
        print(f'  Aplicando transformação vertical para: {image_path}')

        image = cv2.imread(str(image_path))

        with open(label_path, 'r') as f:
            lines = f.readlines()

        bbox = []
        for line in lines:
            l = line.split(' ')
            bbox.append([float(l[1]), float(l[2]), float(l[3]), float(l[4]), str(l[0])])

        transform = A.Compose([A.VerticalFlip()], bbox_params=A.BboxParams(format='yolo'))
        transformed = transform(image=image, bboxes=bbox)

        image_transformed = transformed['image']
        boxes = transformed['bboxes']

        save_image_path = image_path.parent / f'{save_name}_vf.jpg'
        cv2.imwrite(str(save_image_path), image_transformed)

        save_label_path = label_path.parent / f'{save_name}_vf.txt'
        with open(str(save_label_path), 'w') as f:
            for box in boxes:
                f.write(f"{box[-1]} {box[0]} {box[1]} {box[2]} {box[3]}\n")

        print(f'  Transformação concluída para: {image_path}')

    except Exception as e:
        raise RuntimeError(f'Falhou: {save_name}. Erro: {e}')

# Caminhos dos diretórios
image_folder_path = '/content/Comida2.0.v1i/train/images'
annotation_folder_path = '/content/Comida2.0.v1i/train/labels'

# Executar a transformação
transformer(image_folder_path, annotation_folder_path)


In [ ]:
Image_dir = '/content/Comida2.0.v1i/train/images'

num_samples = 9
image_files = os.listdir(Image_dir)

# Randomly select num_samples images
rand_images = random.sample(image_files, num_samples)

fig, axes = plt.subplots(3, 3, figsize=(11, 11))

for i in range(num_samples):
    image = rand_images[i]
    ax = axes[i // 3, i % 3]
    ax.imshow(plt.imread(os.path.join(Image_dir, image)))
    ax.set_title(f'Image {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [7]:
image = cv2.imread("/content/Comida2.0.v1i/train/images/----_jpg.rf.1ddba127dd12741bd457a0c82c39a155_hf.jpg")
h, w, c = image.shape
print(f"The image has dimensions {w}x{h} and {c} channels.")

The image has dimensions 416x416 and 3 channels.


In [ ]:
model = YOLO("yolov8n.pt")

# Use the model to detect object
image = "/content/Comida2.0.v1i/train/images/----_jpg.rf.1ddba127dd12741bd457a0c82c39a155_hf.jpg"
result_predict = model.predict(source = image, imgsz=(416))

# show results
plot = result_predict[0].plot()
plot = cv2.cvtColor(plot, cv2.COLOR_BGR2RGB)
display(Image.fromarray(plot))

In [ ]:
Final_model = YOLO('yolov8n.pt')
Result_Final_model = Final_model.train(data="/content/Comida2.0.v1i/data.yaml", epochs=100, patience=20, batch = 16, cache = True, imgsz=416, iou = 0.5, augment=True, degrees=25.0, fliplr=0.0, lr0=0.0001, optimizer='Adam')

In [ ]:
print('result of training')
tpaths2=[]
for dirname, _, filenames in os.walk('/content/runs/detect/train'):
    for filename in filenames:
        if filename[-4:]=='.png' or filename[-4:]=='.jpg':
            tpaths2+=[(os.path.join(dirname, filename))]
tpaths2=sorted(tpaths2)
print(tpaths2[0])

In [ ]:
for path in tpaths2:
    image = Image.open(path)
    image=np.array(image)
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.show()

In [12]:
print('Predict')
best_path0='/content/runs/detect/train/weights/best.pt'
source0='/content/Comida2.0.v1i/valid/images'

Predict


In [13]:
ppaths=[]
for dirname, _, filenames in os.walk(source0):
    for filename in filenames:
        if filename[-4:]=='.jpg':
            ppaths+=[(os.path.join(dirname, filename))]
ppaths=sorted(ppaths)
print(ppaths[0])

/content/Comida2.0.v1i/valid/images/007_jpg.rf.bdf4efdbad0d7ce195263e3c21856a27.jpg


In [14]:
model2 = YOLO(best_path0)
print('yolo task=detect mode=predict')

yolo task=detect mode=predict


In [ ]:
!yolo task=detect mode=predict model={best_path0} conf=0.4 source={source0}

In [ ]:
print('Result of Prediction')
results = model2.predict(source0,conf=0.4)
print(len(results))

In [ ]:
PBOX=pd.DataFrame(columns=range(6))
for i in range(len(results)):
    arri=pd.DataFrame(results[i].boxes.data).astype(float)
    path=ppaths[i]
    file=path.split('/')[-1]
    arri=arri.assign(file=file)
    arri=arri.assign(i=i)
    PBOX=pd.concat([PBOX,arri],axis=0)
PBOX.columns=['x','y','x2','y2','confidence','class','file','i']
display(PBOX)

In [35]:
class_map = {
    0: 'beet',
    1: 'bell_pepper',
    2: 'cabbage',
    3: 'carrot',
    4: 'cucumber',
    5: 'egg',
    6: 'eggplant',
    7: 'garlic',
    8: 'onion',
    9: 'potato',
    10: 'tomato',
    11: 'zucchini'
}

# Accessing a specific item
print(class_map[0])


beet


In [ ]:
PBOX['class'] = PBOX['class'].apply(lambda x: class_map[int(x)])
PBOX = PBOX.reset_index(drop=True)
display(PBOX)
display(PBOX['class'].value_counts())

In [37]:
def draw_box2(n0):

    ipath=ppaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath.split('/')[-1]

    if PBOX[PBOX['file']==file] is not None:
        box=PBOX[PBOX['file']==file]
        box=box.reset_index(drop=True)
        #display(box)

        for i in range(len(box)):
            label=box.loc[i,'class']
            x=int(box.loc[i,'x'])
            y=int(box.loc[i,'y'])
            x2=int(box.loc[i,'x2'])
            y2=int(box.loc[i,'y2'])
            #print(label,x,y,x2,y2)
            cv2.putText(image, f'{label}', (x, int(y-4)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0),2)
            cv2.rectangle(image,(x,y),(x2,y2),(0,255,0),2) #green

    #plt.imshow(image)
    #plt.show()

    return image

In [38]:
def create_animation(ims):
    fig=plt.figure(figsize=(12,8))
    im=plt.imshow(cv2.cvtColor(ims[0],cv2.COLOR_BGR2RGB))
    text = plt.text(0.05, 0.05, f'Slide {0}', transform=fig.transFigure, fontsize=14, color='blue')
    plt.axis('off')
    plt.close()

    def animate_func(i):
        im.set_array(cv2.cvtColor(ims[i],cv2.COLOR_BGR2RGB))
        text.set_text(f'Slide {20+i}')
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000)

In [39]:
images2=[]
for i in tqdm(range(len(ppaths))):
    images2+=[draw_box2(i)]

  0%|          | 0/213 [00:00<?, ?it/s]

In [ ]:
create_animation(images2)